## Assignment 3: Gnerating Sneaker Designs

In [45]:
# DO NOT REMOVE!
import os

import pickle

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import numpy as np

from datasets import load_dataset

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

import os

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tt

from pytorch_model_summary import summary

In [44]:
!pip install scikit-learn


  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB 435.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/11.0 MB 544.7 kB/s eta 0:00:21
    --------------------------------------- 0.3/11.0 MB 1.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/11.0 MB 2.6 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/11.0 MB 3.5 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/11.0 MB 3.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.4/11.0 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.0 MB 4.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.0 MB 4.2 MB/s eta 0:00:03
   ------- -------------

### Data

In [3]:
# Verify that the Kaggle JSON file is correctly placed
import os

kaggle_json_path = os.path.join(os.path.expanduser('~'), '.kaggle', 'kaggle.json')
if not os.path.exists(kaggle_json_path):
    raise FileNotFoundError(f"{kaggle_json_path} not found. Please ensure the file is placed correctly.")


In [5]:
!pip install kaggle

In [6]:
!kaggle datasets download -d aahashemi/sneaker-image-dataset

Dataset URL: https://www.kaggle.com/datasets/aahashemi/sneaker-image-dataset
License(s): unknown




  0%|          | 0.00/1.09G [00:00<?, ?B/s]
  0%|          | 1.00M/1.09G [00:00<10:45, 1.81MB/s]
  0%|          | 2.00M/1.09G [00:00<07:13, 2.69MB/s]
  0%|          | 3.00M/1.09G [00:01<06:02, 3.22MB/s]
  0%|          | 4.00M/1.09G [00:01<05:55, 3.28MB/s]
  0%|          | 5.00M/1.09G [00:01<05:41, 3.41MB/s]
  1%|          | 6.00M/1.09G [00:01<05:20, 3.62MB/s]
  1%|          | 7.00M/1.09G [00:02<05:36, 3.46MB/s]
  1%|          | 8.00M/1.09G [00:02<05:34, 3.47MB/s]
  1%|          | 9.00M/1.09G [00:02<05:07, 3.77MB/s]
  1%|          | 10.0M/1.09G [00:03<04:58, 3.88MB/s]
  1%|          | 11.0M/1.09G [00:03<05:08, 3.75MB/s]
  1%|1         | 12.0M/1.09G [00:03<05:12, 3.70MB/s]
  1%|1         | 13.0M/1.09G [00:03<05:00, 3.84MB/s]
  1%|1         | 14.0M/1.09G [00:04<04:53, 3.93MB/s]
  1%|1         | 15.0M/1.09G [00:04<04:45, 4.03MB/s]
  1%|1         | 16.0M/1.09G [00:04<04:47, 4.01MB/s]
  2%|1         | 17.0M/1.09G [00:04<04:34, 4.20MB/s]
  2%|1         | 18.0M/1.09G [00:05<04:12, 4.55MB/s]
 

In [9]:
import zipfile
import os

# Define the path to your ZIP file and the directory to extract to
zip_file = 'sneaker-image-dataset.zip'
extract_dir = './data/'

# Check if the ZIP file exists
if not os.path.exists(zip_file):
    raise FileNotFoundError(f"{zip_file} not found. Please ensure the file is in the correct location.")

# Create the extraction directory if it does not exist
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the ZIP file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Dataset extracted to {extract_dir}")


Dataset extracted to ./data/


In [46]:
class Sneakers(Dataset):
    """Dataset class for sneakers images."""

    def __init__(self, root_dir, transforms=None, train_size=0.8, val_size=0.1, test_size=0.1, random_state=42):
        """
        Initialize the dataset.

        Args:
        - root_dir (str): Root directory containing all sneaker image files.
        - transforms (callable, optional): Optional transforms to be applied to the data.
        - train_size (float): Fraction of data to use for training.
        - val_size (float): Fraction of data to use for validation.
        - test_size (float): Fraction of data to use for testing.
        - random_state (int): Random seed for reproducibility.
        """
        self.root_dir = root_dir
        self.transforms = transforms
        
        # List all image files
        self.image_files = os.listdir(self.root_dir)
        self.image_files = [f for f in self.image_files if f.endswith('.jpg') or f.endswith('.png')]  # Filter for image files

        # Split dataset into train, val, and test
        self.train_files, val_test_files = train_test_split(self.image_files, test_size=(val_size + test_size), random_state=random_state)
        self.val_files, self.test_files = train_test_split(val_test_files, test_size=(test_size / (val_size + test_size)), random_state=random_state)

        if len(self.train_files) == 0:
            raise ValueError("No training data found.")
        if len(self.val_files) == 0:
            raise ValueError("No validation data found.")
        if len(self.test_files) == 0:
            raise ValueError("No test data found.")

    def __len__(self):
        return len(self.train_files) + len(self.val_files) + len(self.test_files)

    def __getitem__(self, idx):
        if idx < len(self.train_files):
            file_path = os.path.join(self.root_dir, self.train_files[idx])
            mode = 'train'
        elif idx < len(self.train_files) + len(self.val_files):
            file_path = os.path.join(self.root_dir, self.val_files[idx - len(self.train_files)])
            mode = 'val'
        else:
            file_path = os.path.join(self.root_dir, self.test_files[idx - len(self.train_files) - len(self.val_files)])
            mode = 'test'

        image = Image.open(file_path).convert('RGB')  # Open image and convert to RGB (if not already)

        if self.transforms:
            image = self.transforms(image)

        return image, mode

### GAN

#### Generator

In [34]:
class Generator(nn.Module):
    def __init__(self, generator_net, z_size):
        super(Generator, self).__init__()
        
        # Initialize NN.
        self.generator_net = generator_net
        # find size of latents
        self.z_size = z_size

    def generate(self, z):
        # apply generator net for a given z
        return self.generator_net(z)

    def sample(self, batch_size=16):
        # sample latents
        z = torch.randn(batch_size, self.z_size)
        return self.generate(z)

    def forward(self, z=None):
        if z is None:
            return self.sample()
        else:
            return self.generate(z)

#### Discriminator

In [35]:
class Discriminator(nn.Module):
    def __init__(self, discriminator_net):
        super(Discriminator, self).__init__()
        self.discriminator_net = discriminator_net

    def forward(self, x):
        # Apply NN
        return self.discriminator_net(x)

#### GAN

In [36]:

class GAN(nn.Module):
    def __init__(self, generator, discriminator, epsilon=0.00001):
        super(GAN, self).__init__()

        print('GAN by JT.')
        
        self.generator = generator
        self.discriminator = discriminator
        
        
        self.epsilon = epsilon

    def forward(self, x_real, reduction='avg', mode='discriminator'):
        # The forward pass calculates the adversarial loss.
        if mode == 'generator':
            # For the generator, we first sample FAKE data.
            x_fake_gen = self.generator.sample(x_real.shape[0])

            # Then, we calculate outputs of the discriminator for the FAKE data.
            d_fake = torch.clamp(self.discriminator(x_fake_gen), self.epsilon, 1. - self.epsilon)
            
            # The loss for the generator is log(1 - D(G(z))).
            loss = torch.log(1. - d_fake)

        elif mode == 'discriminator':
            # For the discriminator, we first sample FAKE data.
            x_fake_gen = self.generator.sample(x_real.shape[0])

            # Then, we calculate outputs of the discriminator for the FAKE data.
            # NOTE: We clamp for the numerical stability later on.
            d_fake = torch.clamp(self.discriminator(x_fake_gen), self.epsilon, 1. - self.epsilon)
            
            # Moreover, we calculate outputs of the discriminator for the REAL data.
            # NOTE: We clamp for... the numerical stability (again).
            d_real = torch.clamp(self.discriminator(x_real), self.epsilon, 1. - self.epsilon)

            # The final loss for the discriminator is log(1 - D(G(z))) + log D(x).
            # NOTE: We take the minus sign because we MAXIMIZE the adversarial loss wrt 
            # discriminator, so we MINIMIZE the negative adversarial loss wrt discriminator.
            loss = -(torch.log(d_real) + torch.log(1. - d_fake))
        
        if reduction == 'sum':
            return loss.sum()
        else:
            return loss.mean()

    def sample(self, batch_size=64):
        return self.generator.sample(batch_size=batch_size)

#### Evaluation

In [37]:
def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model')

    model_best.eval()
    loss_gen = 0.
    loss_dis = 0.
    N = 0.
    for indx_batch, test_batch in enumerate(test_loader):
        loss_t_gen = model_best.forward(test_batch, reduction='sum', mode='generator')
        loss_t_dis = model_best.forward(test_batch, reduction='sum', mode='discriminator')
        
        loss_gen = loss_gen + loss_t_gen.item()
        loss_dis = loss_dis + loss_t_dis.item()
        
        N = N + test_batch.shape[0]
    
    loss_gen = loss_gen / N
    loss_dis = loss_dis / N

    if epoch is None:
        print(f'FINAL LOSS: gen={loss_gen}, dis={loss_dis}')
    else:
        print(f'Epoch: {epoch}, val gen={loss_gen}, val dis={loss_dis}')

    return loss_gen, loss_dis


def samples_real(name, test_loader):
    # REAL-------
    num_x = 4
    num_y = 4
    x = next(iter(test_loader)).detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.close()


def samples_generated(name, data_loader, extra_name=''):
    x = next(iter(data_loader)).detach().numpy()

    # GENERATIONS-------
    model_best = torch.load(name + '.model')
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.close()


def plot_curve(name, nll_val, xaxis='epochs', yaxis='nll'):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel(xaxis)
    plt.ylabel(yaxis)
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.close()

Training

In [38]:
def training(name, num_epochs, model, optimizer_gen, optimizer_dis, training_loader, val_loader):
    gen_val = []
    dis_val = []
    best_loss = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, batch in enumerate(training_loader):
            
            # -Discriminator
            loss_dis = model.forward(batch, mode='discriminator')
                        
            optimizer_dis.zero_grad()
            optimizer_gen.zero_grad()
            loss_dis.backward(retain_graph=True)
            optimizer_dis.step()
            
            # -Generator
            loss_gen = model.forward(batch, mode='generator')
            
            optimizer_dis.zero_grad()
            optimizer_gen.zero_grad()
            loss_gen.backward(retain_graph=True)
            optimizer_gen.step()

        # Validation
        loss_val_gen, loss_val_dis = evaluation(val_loader, model_best=model, epoch=e)
        gen_val.append(loss_val_gen)  # save for plotting
        dis_val.append(loss_val_dis)  # save for plotting
        
        torch.save(model, name + '.model')
        samples_generated(name, val_loader, extra_name="_epoch_" + str(e))

    gen_val = np.asarray(gen_val)
    dis_val = np.asarray(dis_val)

    return gen_val, dis_val

In [39]:

transforms_train = tt.Compose([
    tt.ToTensor(),  # Convert to tensor
    tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

transforms_val = tt.Compose([
    tt.ToTensor(),  # Convert to tensor
    tt.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

In [47]:
data_dir = '.\\data\\sneaker-image-dataset'

dataset = Sneakers(root_dir=data_dir, transforms=transforms_train)
